In [1]:
# Install libraries 
!pip install lightning torch==1.10.1 torchaudio==0.10.1 torchvision==0.11.2 pytorch-lightning==1.9.0 lightning-utilities==0.10.1

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/ohpc/pub/apps/python/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


# Architecture of the Classifier

as defined by our Capstone project primary paper "PyTorch-Wildlife" GitHub repository.

References:
- https://github.com/microsoft/CameraTraps/blob/main/PW_FT_classification/src/models/plain_resnet.py
- https://github.com/microsoft/CameraTraps/blob/main/PW_FT_classification/src/algorithms/plain.py

In [2]:
# Import necessary libraries.
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import pytorch_lightning as pl
from torchvision.models.resnet import Bottleneck, ResNet
from torchvision.models.resnet import *

# Try importing a function to load pre-trained weights; if not available, use an alternative.
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_state_dict_from_url

# Exportable class names for external use
__all__ = [
    'PlainResNetClassifier',  # Custom ResNet classifier
    'Plain'  # PyTorch Lightning model wrapper
]

# Pre-trained model URL for initializing ResNet-50 with pretrained weights (ImageNet).
model_urls = {
    'resnet50': 'https://download.pytorch.org/models/resnet50-0676ba61.pth'
}

class ResNetBackbone(ResNet):
    """
    Custom ResNet backbone class for feature extraction.
    Inherits from the original ResNet class to use it as a feature extractor.
    """

    def __init__(self, block, layers, zero_init_residual=False,
                 groups=1, width_per_group=64,
                 replace_stride_with_dilation=None,
                 norm_layer=None):
        """
        Initialize the ResNet backbone.
        Args:
            - block: The building block used in ResNet (e.g., Bottleneck).
            - layers: Number of layers in each stage.
            - zero_init_residual: If True, zero-initialize the residual connection.
            - groups, width_per_group: Controls for ResNet's internal structure.
            - replace_stride_with_dilation: Dilation to replace strides.
            - norm_layer: The normalization layer to use.
        """
        super(ResNetBackbone, self).__init__(
            block=block,
            layers=layers,
            zero_init_residual=zero_init_residual,
            groups=groups,
            width_per_group=width_per_group,
            replace_stride_with_dilation=replace_stride_with_dilation,
            norm_layer=norm_layer,
        )

    def _forward_impl(self, x):
        """
        Forward pass implementation for the ResNet backbone.
        """
        # Apply initial convolution, batch norm, and max pooling.
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Pass through each ResNet layer sequentially.
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Apply global average pooling and flatten the tensor.
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

class PlainResNetClassifier(nn.Module):
    """
    Custom ResNet classifier class that utilizes the ResNet backbone.
    """

    name = 'PlainResNetClassifier'

    def __init__(self, num_cls=2, num_layers=50):
        """
        Initialize the PlainResNetClassifier.
        Args:
            - num_cls: Number of classes to classify.
            - num_layers: Number of layers in the ResNet (e.g., 50 for ResNet-50).
        """
        super(PlainResNetClassifier, self).__init__()
        self.num_cls = num_cls
        self.num_layers = num_layers
        self.feature = None  # Backbone for feature extraction.
        self.classifier = None  # Linear layer for classification.
        self.criterion_cls = None  # Loss criterion.

        # Initialize the network architecture and load pre-trained weights.
        self.setup_net()

    def setup_net(self):
        """
        Set up the ResNet network and initialize its weights.
        """
        kwargs = {}  # Optional parameters for ResNet configuration.

        # Choose architecture based on specified layer count.
        if self.num_layers == 50:
            block = Bottleneck  # Use Bottleneck block for ResNet-50.
            layers = [3, 4, 6, 3]  # Standard layer configuration for ResNet-50.
            # Load pre-trained ResNet-50 weights.
            self.pretrained_weights = load_state_dict_from_url(
                model_urls['resnet50'], progress=True)
        else:
            raise Exception('ResNet Type not supported.')

        # Construct the feature extractor and classifier.
        self.feature = ResNetBackbone(block, layers, **kwargs)
        self.classifier = nn.Linear(512 * block.expansion, self.num_cls)

    def setup_criteria(self):
        """
        Set up the criterion (loss function) for classification.
        """
        self.criterion_cls = nn.CrossEntropyLoss()

class Plain(pl.LightningModule):
    """
    Defines the architecture for training a model using PyTorch Lightning.
    """

    name = 'Plain'

    def __init__(self, conf, train_class_counts, id_to_labels, **kwargs):
        """
        Initializes the Plain model.
        Args:
            - conf: Configuration object containing training parameters.
            - train_class_counts: List of class counts for training.
            - id_to_labels: Dictionary mapping class indices to labels.
        """
        super().__init__()
        # Store hyperparameters from the configuration object.
        self.hparams.update(conf.__dict__)
        self.save_hyperparameters(ignore=['conf', 'train_class_counts'])
        self.train_class_counts = train_class_counts  # Store class counts.
        self.id_to_labels = id_to_labels  # Store class label mappings.
        # Initialize the custom ResNet-based classifier.
        self.net = PlainResNetClassifier(num_cls=self.hparams.num_classes,
                                          num_layers=self.hparams.num_layers)

    def configure_optimizers(self):
        """
        Configures the optimizers and learning rate schedulers.
        """
        # Define parameters for the optimizer, separating feature and classifier parameters.
        net_optim_params_list = [
            # Feature extraction parameters.
            {'params': self.net.feature.parameters(),
             'lr': self.hparams.lr_feature,
             'momentum': self.hparams.momentum_feature,
             'weight_decay': self.hparams.weight_decay_feature},
            # Classifier parameters.
            {'params': self.net.classifier.parameters(),
             'lr': self.hparams.lr_classifier,
             'momentum': self.hparams.momentum_classifier,
             'weight_decay': self.hparams.weight_decay_classifier}
        ]
        # Setup optimizer and learning rate scheduler.
        optimizer = torch.optim.SGD(net_optim_params_list)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=self.hparams.step_size, gamma=self.hparams.gamma)
        return [optimizer], [scheduler]

    def on_train_start(self):
        """
        Hook function called at the start of training.
        Initializes best accuracy and sets up the loss function.
        """
        self.net.setup_criteria()  # Set up the loss function for classification.

    def training_step(self, batch, batch_idx):
        """
        Training step for each batch.

        Args:
            batch: The current batch of data containing inputs and labels.
            batch_idx: The index of the current batch.

        Returns:
            Tensor: The loss for the current training step.
        """
        data, label_ids = batch[0], batch[1]  # Unpack batch data into features and labels.

        # Forward pass through feature extractor and classifier.
        feats = self.net.feature(data)  # Extract features from the input data.
        logits = self.net.classifier(feats)  # Classify the extracted features.

        # Calculate loss using the criterion (cross-entropy loss between predictions and true labels).
        loss = self.net.criterion_cls(logits, label_ids)

        return loss

    def on_validation_start(self):
        """
        Hook function called at the start of validation.
        Initializes storage for validation outputs.
        """
        self.val_st_outs = []

    def validation_step(self, batch, batch_idx):
        """
        Validation step for each batch.

        Args:
            batch: The current batch of data containing inputs and labels.
            batch_idx: The index of the current batch.
        """
        data, label_ids = batch[0], batch[1]  # Unpack batch data into features and labels.

        # Forward pass through the network for validation.
        feats = self.net.feature(data)  # Extract features from the validation data.
        logits = self.net.classifier(feats)  # Classify the extracted features.
        preds = logits.argmax(dim=1)  # Get the predicted class labels.

        # Append predictions and true labels to the validation output list.
        self.val_st_outs.append((preds.detach().cpu().numpy(),  # Store predictions.
                                 label_ids.detach().cpu().numpy()))  # Store true labels.

    def on_test_start(self):
        """
        Hook function called at the start of testing.
        Initializes storage for test outputs.
        """
        self.te_st_outs = []

    def test_step(self, batch, batch_idx):
        """
        Test step for each batch.

        Args:
            batch: The current batch of data, including metadata.
            batch_idx: The index of the current batch.
        """
        data, label_ids, labels, file_ids = batch  # Unpack batch data (features, true labels, additional metadata).

        # Forward pass through the network for testing.
        feats = self.net.feature(data)  # Extract features from the test data.
        logits = self.net.classifier(feats)  # Classify the extracted features.
        preds = logits.argmax(dim=1)  # Get the predicted class labels.

        # Append predictions and relevant data to the test output list.
        self.te_st_outs.append((preds.detach().cpu().numpy(),  # Store predictions.
                               label_ids.detach().cpu().numpy(),  # Store true labels.
                               feats.detach().cpu().numpy(),  # Store features for analysis.
                               logits.detach().cpu().numpy(),  # Store raw logits for further inspection.
                               labels, file_ids))  # Store additional metadata.

    def on_predict_start(self):
        """
        Hook function called at the start of prediction.
        Initializes storage for prediction outputs.
        """
        self.pr_st_outs = []

    def predict_step(self, batch, batch_idx):
        """
        Prediction step for each batch.

        Args:
            batch: The current batch of data, including metadata.
            batch_idx: The index of the current batch.
        """
        data, file_ids = batch  # Unpack batch data into features and metadata.

        # Forward pass through the network for predictions.
        feats = self.net.feature(data)  # Extract features from the input data.
        logits = self.net.classifier(feats)  # Classify the extracted features.
        preds = logits.argmax(dim=1)  # Get the predicted class labels.

        # Append predictions and relevant data to the prediction output list.
        self.pr_st_outs.append((preds.detach().cpu().numpy(),  # Store predictions.
                               feats.detach().cpu().numpy(),  # Store extracted features.
                               logits.detach().cpu().numpy(),  # Store raw logits for analysis.
                               file_ids))  # Store file IDs for reference.

### Initialize the model with the configurations

In [3]:
import os

# Define the path to the trained ResNet-50 checkpoint file
checkpoint_path = 'weights/Crop/Plain/Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt'

# Load the checkpoint file
checkpoint = torch.load(checkpoint_path)

# Accessing the Hyperparameters used for training the classifier
hyper_parameters = checkpoint['hyper_parameters']

print("\nHyperparameters:")
print(hyper_parameters)

/tmp/ipykernel_5050/2461769364.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)



Hyperparameters:
{'conf_id': 'Crop_res50_plain_082723', 'algorithm': 'Plain', 'log_dir': 'Crop', 'num_epochs': 15, 'log_interval': 10, 'parallel': 0, 'dataset_root': './data/imgs', 'dataset_name': 'Custom_Crop', 'annotation_dir': './data/imgs', 'split_path': './data/imgs/tiger_binary.csv', 'test_size': 0.2, 'val_size': 0.2, 'split_data': True, 'split_type': 'random', 'batch_size': 32, 'num_workers': 8, 'num_classes': 2, 'model_name': 'PlainResNetClassifier', 'num_layers': 50, 'weights_init': 'ImageNet', 'lr_feature': 0.01, 'momentum_feature': 0.9, 'weight_decay_feature': 0.0005, 'lr_classifier': 0.01, 'momentum_classifier': 0.9, 'weight_decay_classifier': 0.0005, 'step_size': 10, 'gamma': 0.1, 'evaluate': None, 'val': False, 'predict': False, 'predict_root': '', 'id_to_labels': {0: 'Not Tiger', 1: 'Tiger'}}


In [4]:
# Configuration class to specify model parameters.
class Config:
    def __init__(self):
        # Hyperparameters extracted from the Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt checkpoint file
        self.conf_id = 'Crop_res50_plain_082723'  # Configuration ID
        self.algorithm = 'Plain'  # Algorithm name
        self.log_dir = 'Crop'  # Directory for logs
        self.num_epochs = 15  # Number of epochs
        self.log_interval = 10  # Interval for logging
        self.parallel = 0  # Parallel processing flag
        self.dataset_root = './data/imgs'  # Root directory of the dataset
        self.dataset_name = 'Custom_Crop'  # Name of the dataset
        self.annotation_dir = './data/imgs'  # Directory for annotations
        self.split_path = './data/imgs/tiger_binary.csv'  # Path to split file
        self.test_size = 0.2  # Proportion of data to use for testing
        self.val_size = 0.2  # Proportion of data to use for validation
        self.split_data = True  # Flag indicating whether to split data
        self.split_type = 'random'  # Method for splitting the dataset
        self.batch_size = 32  # Batch size for training
        self.num_workers = 8  # Number of workers for data loading
        self.num_classes = 2  # Number of classes in the classification task
        self.model_name = 'PlainResNetClassifier'  # Name of the model
        self.num_layers = 50  # Number of layers in the ResNet model
        self.weights_init = 'ImageNet'  # Weight initialization strategy
        self.lr_feature = 0.01  # Learning rate for feature extractor
        self.momentum_feature = 0.9  # Momentum for feature extractor optimizer
        self.weight_decay_feature = 0.0005  # Weight decay for feature extractor
        self.lr_classifier = 0.01  # Learning rate for classifier
        self.momentum_classifier = 0.9  # Momentum for classifier optimizer
        self.weight_decay_classifier = 0.0005  # Weight decay for classifier
        self.step_size = 10  # Step size for learning rate scheduler
        self.gamma = 0.1  # Learning rate decay factor
        self.evaluate = None  # Evaluation flag
        self.val = False  # Validation flag
        self.predict = False  # Prediction flag
        self.predict_root = ''  # Directory for prediction data
        self.id_to_labels = {0: 'Not Tiger', 1: 'Tiger'}  # Mapping from class IDs to class labels

# Initialize the model with the configuration.
conf = Config()
train_class_counts = [3458, 4440]  # Example class counts for training.
id_to_labels = conf.id_to_labels  # Use the configuration's label mapping.

# Initialize model
model = Plain(conf, train_class_counts, id_to_labels)
# Print the model architecture.
print(model)

Plain(
  (net): PlainResNetClassifier(
    (feature): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

### Initialize the model with the configurations and load the weights from the checkpoint

In [5]:
# Load the checkpoint into the model.
def load_model_from_checkpoint(checkpoint_path, conf):
    """
    Load the model from the checkpoint file.
    Args:
        - checkpoint_path: Path to the checkpoint file.
        - conf: Configuration object.
    """
    # Use the class method to load from checkpoint.
    model = Plain.load_from_checkpoint(checkpoint_path, conf=conf,
                                       train_class_counts=[3458, 4440], 
                                       id_to_labels=conf.id_to_labels)
    
    model.eval()  # Set the model to evaluation mode
    return model

# Initialize the model with configuration and load the checkpoint.
conf = Config()

# Initialize and load model
model = load_model_from_checkpoint(checkpoint_path, conf)
# Print the loaded model.
print(model)

Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint weights/Crop/Plain/Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt`


Plain(
  (net): PlainResNetClassifier(
    (feature): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

#### Classifier model test

In [6]:
from torchvision import transforms
from PIL import Image

# Function to preprocess the input image
def preprocess_image(image_path):
    # Define the transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match the input size of the model
        transforms.ToTensor(),           # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
    ])
    
    # Load the image
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB mode
    image = transform(image)  # Apply the transformations
    image = image.unsqueeze(0)  # Add a batch dimension
    return image

# Function to predict the class of the image
def predict(model, image_tensor, device):
    image_tensor = image_tensor.to(device)  # Move the image tensor to the same device as the model
    with torch.no_grad():  # Disable gradient calculation
        logits = model.net.classifier(model.net.feature(image_tensor))  # Forward pass
        preds = torch.argmax(logits, dim=1)  # Get the predicted class
    return preds.item()

In [7]:
image_path = 'data/imgs/Leopard (1).jpg'

# Determine the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Preprocess the image
image_tensor = preprocess_image(image_path)

# Make prediction
predicted_class = predict(model, image_tensor, device)

# Map the predicted class to the label
predicted_label = conf.id_to_labels[predicted_class]
print(f"Predicted Class: {predicted_class}, Label: {predicted_label}")

Predicted Class: 0, Label: Not Tiger


In [8]:
image_path = 'data/imgs/Tiger (531).jpg'

# Determine the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Preprocess the image
image_tensor = preprocess_image(image_path)

# Make prediction
predicted_class = predict(model, image_tensor, device)

# Map the predicted class to the label
predicted_label = conf.id_to_labels[predicted_class]
print(f"Predicted Class: {predicted_class}, Label: {predicted_label}")

Predicted Class: 1, Label: Tiger
